In [1]:
import io
import socket
import struct  
import logging
import socketserver
import threading 
from http import server

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('0.0.0.0', 8000))  

fmt = '<L' # unsigned_long_int
len_fmt = struct.calcsize(fmt) 


class StreamingOutput(object):
    def __init__(self):
        self.frame = None
        self.buffer = io.BytesIO()
        self.condition = threading.Condition()
        self.receive_thread = threading.Thread(target=self.receiver)
        self.receive_thread.daemon = True
        self.receive_thread.start() 
        
    def receiver(self):
        while True: 
            len_img, = struct.unpack(fmt, server_socket.recv(len_fmt)) 
            img_IO = io.BytesIO()
            while len_img: 
                len_img -= 65507  
                img_IO.write(server_socket.recv(65507)) # 65507 is the max length of UDP Packet
                if len_img < 0 : break   
            img_IO.seek(0)  
            with self.condition:
                self.frame = img_IO.read()
                self.condition.notify_all()  

class StreamingHandler(server.BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(301)
            self.send_header('Location', '/stream.mjpg')
            self.end_headers() 
        elif self.path == '/stream.mjpg':
            self.send_response(200)
            self.send_header('Age', 0)
            self.send_header('Cache-Control', 'no-cache, private')
            self.send_header('Pragma', 'no-cache')
            self.send_header('Content-Type', 'multipart/x-mixed-replace; boundary=FRAME')
            self.end_headers()
            try:
                while True:
                    with output.condition:
                        output.condition.wait()
                        frame = output.frame
                    self.wfile.write(b'--FRAME\r\n')
                    self.send_header('Content-Type', 'image/jpeg')
                    self.send_header('Content-Length', len(frame))
                    self.end_headers()
                    self.wfile.write(frame)
                    self.wfile.write(b'\r\n')
            except Exception as e:
                logging.warning(
                    'Removed streaming client %s: %s',
                    self.client_address, str(e))
        else:
            self.send_error(404)
            self.end_headers()

class StreamingServer(socketserver.ThreadingMixIn, server.HTTPServer):
    allow_reuse_address = True
    daemon_threads = True

output = StreamingOutput()  

def streaming():
    server = StreamingServer(('', 8001), StreamingHandler)
    server.serve_forever() 
t = threading.Thread(target=streaming)
t.daemon = True
t.start() 

### Run IFrame after client starts sending images

In [2]:
from IPython.display import IFrame    
IFrame('http://192.168.0.94:8001', width=800, height=650)

192.168.0.123 - - [26/Aug/2019 05:50:25] "GET /stream.mjpg HTTP/1.1" 200 -
